# Problema de la Mochila / Knapsack Problem (KP)

En algoritmia, el problema de la mochila, comúnmente abreviado por KP (del inglés *Knapsack Problem*) es un problema de optimización combinatoria, es decir, que busca la mejor solución entre un conjunto finito de posibles soluciones a un problema. Modela una situación análoga a llenar una mochila, incapaz de soportar más de un peso determinado, con todo o parte de un conjunto de objetos, cada uno con un peso y valor específicos. Los objetos colocados en la mochila deben maximizar el valor total sin exceder el peso máximo.

Este problema se presenta en uno de los libros de la bibliografía recomendada, *Fundamentals of algorithmics. Chapter 6: Greedy algorithms (Brassard, G.; Bratley, P.; 1996)*, y propone un algoritmo voraz para resolver el caso en el que podemos fraccionar los objetos. Si no fuera así (en el caso del problema de la mochila *discreto*), este algoritmo podría no darnos una solución óptima y tendríamos que requerir a otra estrategia, como programación dinámica.

Vamos a implementar dicho algoritmo. Este podría ser su pseudocódigo:

<p align="center">
  <img src="imgs/psc_knapsack_tfg.png">
</p>

Para que a cada paso del greedy loop el primer paso sea trivial, vamos a ordenar la lista de pesos primero tal y como corresponde, para luego ir escogiendo secuencialmente los objetos. Ordenamos por **valor por unidad de peso**, de modo decreciente (esto determinará cuál es el *mejor objeto restante*):

In [1]:
import numpy as np

'''
 Función que ordena la primera lista según su cociente con la
 segunda (i.e., por l2[i]/l1[i]), de modo decreciente.
 
 Entrada:
     l1: list -> lista de tamaño n
     l2: list -> lista de tamaño n
 Salida:
     list -> Lista de pares [pos, val] donde pos es la posición original
             del valor en l1 y val es dicho valor. 
             Devuelve None en caso de error.
'''
def ordenar_lista_cociente(l1: list, l2: list) -> list:
    n = len(l1)
    if n != len(l2):
        return None
    lcocientes = np.array([l2[i]/l1[i] for i in range(n)])
    np_l1 = np.array(list(enumerate(l1,0)))
    return np_l1[lcocientes.argsort()[::-1]].tolist()

In [2]:
'''
 Función que selecciona el objeto óptimo para la mochila,
 lo extrae de la lista y lo devuelve
 
 Entrada:
     lista: list -> lista de tamaño n ya ordenada
 Salida:
     list -> lista [pos, val] de la primera posición de
             la lista de entrada
'''
def seleccionar(lista: list) -> list:
    return lista.pop(0)

In [3]:
'''
 Algoritmo que resuelve de manera óptima el problema de la mochila
 
 Entrada:
     p: list -> lista de tamaño n con los pesos de los objetos
     v: list -> lista de tamaño n con los valores de los objetos
     peso_max: int -> peso máximo que puede contener la mochila
 Salida:
     (list, float) -> Tupla de la lista con la cantidad óptima 
                     (de 0 a 1) de cada objeto en la mochila y
                     el valor total de la mochila.
                     Devuelve None en caso de error.
'''
def mochila(p: list, v: list, peso_max: float) -> (list, float):
    if peso_max <= 0:
        print("Error: El peso máximo debe ser positivo")
        return None
    
    p_ordenada = ordenar_lista_cociente(p, v)
    if p_ordenada is None:
        print("Error: Debe haber el mismo número de pesos y de valores")
        return None
    
    n = len(p_ordenada)
    x = [0] * n
    peso_actual = 0
    while peso_actual < peso_max:
        peso = seleccionar(p_ordenada)
        if peso_actual + peso[1] <= peso_max:
            x[int(peso[0])] = 1
            peso_actual += peso[1]
        else:
            x[int(peso[0])] = (peso_max - peso_actual) / peso[1]
            peso_actual = peso_max
    
    # Hacemos el producto escalar entre la estrategia óptima y los valores
    valor_total = np.array(x) @ np.array(v)  
    
    return x, valor_total

Probamos el algoritmo con el ejemplo del libro

In [4]:
w = [10,20,30,40,50]
v = [20,30,66,40,60]
W = 100

mochila(w,v,W)

([1, 1, 1, 0, 4/5], 164.0)

Ahora un par de casos erróneos para comprobar el control de errores añadido.

In [5]:
mochila(w,[1,2],-2); mochila(w,[1,2],2)

Error: El peso máximo debe ser positivo
Error: Debe haber el mismo número de pesos y de valores


In [6]:
# Un caso con valores decimales que escogí arbitrariamente
w2 = [10,32,22,30.5,10,17,23.6]
v2 = [20,3.4,11,90.1,8,10.2,2]
W2 = 50.7

mochila(w2,v2,W2)

([1, 0, 0, 1, 1, 0.0117647058823531, 0], 118.22)

Ahora vamos a implementar la versión donde usamos **max heaps** para representar los objetos, con el mayor valor por unidad de peso en la raíz. Como la librería *heapq* de Python implementa min heaps, vamos a negar los valores por unidad de peso para conseguir el max heap. De este modo, el *seleccionar()* será la raíz del heap.

In [7]:
import heapq as hq

'''
 Función que crea un max heap según el cociente de los valores
 de la primera lista y la segunda (i.e., por l2[i]/l1[i])
 
 Entrada:
     l1: list -> lista de tamaño n
     l2: list -> lista de tamaño n
 Salida:
     list -> heap (como una lista) de tuplas (c, (pos, val)) 
             donde c es el cociente l2[i]/l1[i], pos es la 
             posición original del valor en l1 y val es dicho valor. 
             Devuelve None en caso de error.
'''
def crear_max_heap(l1: list, l2: list) -> list:
    n = len(l1)
    if n != len(l2):
        return None
    lcocientes = [-l2[i]/l1[i] for i in range(n)]
    ltuplas = list(zip(lcocientes, enumerate(l1, 0)))
    heap = []
    for tupla in ltuplas:
        hq.heappush(heap, tupla)
    return heap

In [8]:
'''
 Función que selecciona el objeto óptimo para la mochila,
 lo extrae del heap y lo devuelve
 
 Entrada:
     heap: list -> lista de tamaño n con estructura de max heap
 Salida:
     tuple -> tupla (pos, val) de la raíz del heap
'''
def seleccionar_heaps(heap: list) -> tuple:
    return hq.heappop(heap)[1]

In [9]:
'''
 Algoritmo que resuelve de manera óptima el problema de la mochila
 usando heaps para representar los objetos
 
 Entrada:
     p: list -> lista de tamaño n con los pesos de los objetos
     v: list -> lista de tamaño n con los valores de los objetos
     peso_max: int -> peso máximo que puede contener la mochila
 Salida:
     (list, float) -> Tupla de la lista con la cantidad óptima 
                     (de 0 a 1) de cada objeto en la mochila y
                     el valor total de la mochila.
                     Devuelve None en caso de error.
'''
def mochila_heaps(p: list, v: list, peso_max: float) -> (list, float):
    if peso_max <= 0:
        print("Error: El peso máximo debe ser positivo")
        return None
    
    heap = crear_max_heap(p, v)
    if heap is None:
        print("Error: Debe haber el mismo número de pesos y de valores")
        return None
    
    n = len(heap)
    x = [0] * n
    peso_actual = 0
    while peso_actual < peso_max:
        peso = seleccionar_heaps(heap)
        if peso_actual + peso[1] <= peso_max:
            x[int(peso[0])] = 1
            peso_actual += peso[1]
        else:
            x[int(peso[0])] = (peso_max - peso_actual) / peso[1]
            peso_actual = peso_max
    
    # Hacemos el producto escalar entre la estrategia óptima y los valores
    valor_total = np.array(x) @ np.array(v)  
    
    return x, valor_total

Volvemos a probar el algoritmo con el ejemplo del libro y el de decimales

In [10]:
w = [10,20,30,40,50]
v = [20,30,66,40,60]
W = 100

w2 = [10,32,22,30.5,10,17,23.6]
v2 = [20,3.4,11,90.1,8,10.2,2]
W2 = 50.7

show(mochila_heaps(w,v,W))
show(mochila_heaps(w2,v2,W2))

([1, 1, 1, 0, 4/5], 164.0)

([1, 0, 0, 1, 1, 0.0117647058823531, 0], 118.22)

Vamos a hacer una pequeña comparación de tiempos entre ambos modos de implementar el algoritmo

In [11]:
import timeit as t

# Encapsulamos los algoritmos en unas funciones sin argumentos para usar luego el módulo timeit
def test_mochila_1():
    mochila(w,v,W)
    
def test_mochila_heaps_1():
    mochila_heaps(w,v,W)
    
def test_mochila_2():
    mochila(w2,v2,W2)
    
def test_mochila_heaps_2():
    mochila_heaps(w2,v2,W2)

Vamos a realizar por ahora solo 10000 pruebas donde ejecutamos 100 veces cada algoritmo, para hacer una estadística rápida (también mediremos el tiempo de ejecución completo de la batería de pruebas con el *magic command* **%time**, para una primera comparación)

In [12]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda son unos 45 segundos
%time tiempos_mochila_1 = t.repeat(test_mochila_1, repeat=10000, number=100)

CPU times: user 43.9 s, sys: 140 ms, total: 44 s
Wall time: 45 s


In [13]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda son unos 25 segundos
%time tiempos_mochila_heaps_1 = t.repeat(test_mochila_heaps_1, repeat=10000, number=100)

CPU times: user 26 s, sys: 63 ms, total: 26 s
Wall time: 26.1 s


In [14]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda es de 1 minuto
%time tiempos_mochila_2 = t.repeat(test_mochila_2, repeat=10000, number=100)

CPU times: user 1min 3s, sys: 47 ms, total: 1min 3s
Wall time: 1min 3s


In [15]:
# Ejecutado en mi ordenador, el tiempo aproximado de la celda es de 1 minuto
%time tiempos_mochila_heaps_2 = t.repeat(test_mochila_heaps_2, repeat=10000, number=100)

CPU times: user 1min 6s, sys: 47 ms, total: 1min 6s
Wall time: 1min 6s


Imprimamos solo los 10 primeros resultados de cada test, por mostrar el orden de los tiempos:

In [16]:
show("Tiempos mochila 1", tiempos_mochila_1[:10])
show("Tiempos mochila heaps 1", tiempos_mochila_heaps_1[:10])
show("Tiempos mochila 2", tiempos_mochila_1[:10])
show("Tiempos mochila heaps 2", tiempos_mochila_heaps_2[:10])

'Tiempos mochila 1' [0.006472299981396645,
 0.005646999983582646,
 0.006760400021448731,
 0.004394699994008988,
 0.004737999988719821,
 0.004496800014749169,
 0.004525199998170137,
 0.004293199977837503,
 0.004347299982327968,
 0.0044371000258252025]

'Tiempos mochila heaps 1' [0.002895899990107864,
 0.0030821000109426677,
 0.00345949997426942,
 0.0030943999881856143,
 0.0027413000352680683,
 0.002756799978669733,
 0.0027108999784104526,
 0.004012499994132668,
 0.0045479999971576035,
 0.003162400040309876]

'Tiempos mochila 2' [0.006472299981396645,
 0.005646999983582646,
 0.006760400021448731,
 0.004394699994008988,
 0.004737999988719821,
 0.004496800014749169,
 0.004525199998170137,
 0.004293199977837503,
 0.004347299982327968,
 0.0044371000258252025]

'Tiempos mochila heaps 2' [0.007798800012096763,
 0.007737600011751056,
 0.007193700002972037,
 0.006779399991501123,
 0.006825599994044751,
 0.006491799955256283,
 0.00696510000852868,
 0.006443299993406981,
 0.006581399997230619,
 0.006545000011101365]

Ahora comparemos los tiempos dependiendo de si hemos usado heaps o no

In [17]:
heaps = 0
for i in range(len(tiempos_mochila_1)):
    if tiempos_mochila_1[i] > tiempos_mochila_heaps_1[i]:
        heaps += 1

show(f"En la primera prueba hay {heaps} veces que es mejor usar heaps y {i-heaps+1} que no")

'En la primera prueba hay 9976 veces que es mejor usar heaps y 24 que no'

In [18]:
heaps = 0
for i in range(len(tiempos_mochila_2)):
    if tiempos_mochila_2[i] > tiempos_mochila_heaps_2[i]:
        heaps += 1

show(f"En la segunda prueba hay {heaps} veces que es mejor usar heaps y {i-heaps+1} que no")

'En la segunda prueba hay 1801 veces que es mejor usar heaps y 8199 que no'

### En conclusión, parece que en el primer caso, el ejemplo del libro, casi siempre es óptimo usar heaps (aproximadamente en un 99% de los casos), pero en el segundo (con números arbitrarios) solo alrededor del 15-20% de las veces. De todos modos estas pruebas deberían realizarse con vectores más aleatorios, de distintos tamaños y mayor número de ejecuciones